### 10 Folium/Altair Maps for Geospatial Analytics in Minutes

Practical, copy-paste map recipes that turn raw lat/longs and polygons into decisions — without becoming a GIS expert.

In [ ]:
# Setup & Tiny Dataset (once)

import pandas as pd
import numpy as np

# 1) Random points roughly around Bengaluru
rng = np.random.default_rng(42)
N = 1000
df_points = pd.DataFrame({
    "lat": 12.9 + rng.normal(0, 0.12, N),
    "lon": 77.6 + rng.normal(0, 0.12, N),
    "value": rng.integers(1, 100, N),
    "category": rng.choice(["A","B","C"], N, p=[0.5,0.3,0.2])
})

# 2) Minimal GeoJSON-like polygon (pretend districts; keep small)
districts = {
  "type":"FeatureCollection",
  "features":[
    {"type":"Feature","properties":{"name":"North"},"geometry":{"type":"Polygon","coordinates":[[[77.45,13.02],[77.9,13.02],[77.9,13.25],[77.45,13.25],[77.45,13.02]]]}},
    {"type":"Feature","properties":{"name":"South"},"geometry":{"type":"Polygon","coordinates":[[[77.45,12.55],[77.9,12.55],[77.9,12.9],[77.45,12.9],[77.45,12.55]]]}},
  ]
}